**Project context** <br>This notebook is part of the project "Bookmakerspy", a project that aims at predicting football games results in the context of the English Premier League (based on data from 2014 to 2018) and ultimately beat bookmakers' odds.<br>
The current notebook "bookmakerspy_preprocessing_dataviz" is the second in a series of 4 notebooks. It uses the dataset output by the first notebook "bookmakerspy_data_collection" and it is followed by "bookmakerspy_modelisation" and "bookmakerspy_odds_strategy".

**Notebook goal**<br>
This notebook aims at pre-processing the "df_stats_odds" dataframe by grouping or creating variables, and at exploring the data through data visualizations so as to emphasize patterns and make assumptions about potential strategies for creating a model. We have iterated over these two steps: pre-processing and data visualizations over time, meaning that some initial exploration with data visualizations has enabled us to refine the data processing steps and we have gone back and forth between these two steps. They should consequently not be approached as a linear process but more as a dynamic process where data visualization and pre-processing steps helped gaining a better understanding of the data overall.



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('df_stats_odds.csv')

In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 142 columns):
 #    Column                         Dtype  
---   ------                         -----  
 0    Unnamed: 0                     int64  
 1    match_id                       int64  
 2    home_team_id                   int64  
 3    home_team_name                 object 
 4    home_team_rating               float64
 5    date                           object 
 6    home_att_goal_low_left         float64
 7    home_won_contest               float64
 8    home_possession_percentage     float64
 9    home_total_throws              float64
 10   home_att_miss_high_left        float64
 11   home_blocked_scoring_att       float64
 12   home_total_scoring_att         float64
 13   home_att_sv_low_left           float64
 14   home_total_tackle              float64
 15   home_att_miss_high_right       float64
 16   home_aerial_won                float64
 17   home_att_miss_right            

1. Removing variables that won't be used for the analysis
After a first observation, given that there is a large number of variables, we decided the remove the variables that we considered as too fine-grained, and that are already included in other more general variables.

In [5]:
df = df.drop(['home_att_goal_low_left','home_att_goal_low_centre','home_att_goal_high_left','home_att_goal_low_right','home_att_goal_high_right','home_att_goal_high_centre'],axis=1)
df = df.drop(['away_att_goal_low_left','away_att_goal_low_centre','away_att_goal_high_left','away_att_goal_low_right','away_att_goal_high_right','away_att_goal_high_centre'],axis=1)
df = df.drop(['home_att_miss_high_left','home_att_miss_high_right','home_att_miss_right','home_att_miss_left','home_att_miss_high'],axis=1)
df = df.drop(['away_att_miss_high_left','away_att_miss_high_right','away_att_miss_right','away_att_miss_left','away_att_miss_high'],axis=1)
df = df.drop(['home_att_sv_low_left','home_att_sv_low_centre','home_att_sv_low_right','home_att_sv_high_left','home_att_sv_high_centre','home_att_sv_high_right'],axis=1)
df = df.drop(['away_att_sv_low_left','away_att_sv_low_centre','away_att_sv_low_right','away_att_sv_high_left','away_att_sv_high_centre','away_att_sv_high_right'],axis=1)
df = df.drop(['home_att_post_high','home_att_post_right','home_att_post_left'],axis=1)
df = df.drop(['away_att_post_high','away_att_post_right','away_att_post_left'],axis=1)
df = df.drop(['home_fk_foul_lost','away_fk_foul_lost'],axis=1)

2. Removing odds not used in the context of the analysis
As we are focusing on the match results (Home, Draw or away) we removed all odds-related variables that are irrelevant in this context

In [6]:
# the following variables concern methodology
df = df.drop(['Bb1X2','BbOU','BbAH'],axis=1)
# odds that are not focused on match result itself
df = df.drop(['BbMxH','BbAvH','BbMxD','BbAvD','BbMxA','BbAvA','BbAv>2.5','BbAv<2.5','BbAHh','BbMxAHH','BbAvAHH','BbMxAHA','BbAvAHA'],axis=1)


3. Grouping variables
We decided to combine the variables accurate pass and total pass to have a ratio

In [7]:
df['home_pass'] =  df['home_accurate_pass'] / df['home_total_pass']
df['away_pass'] =  df['away_accurate_pass'] / df['away_total_pass']

In [9]:
# we can them remove the original variables
df = df.drop(['home_accurate_pass', 'home_total_pass', 'away_accurate_pass', 'away_total_pass'], axis=1)

4. Organising the dataframe
For facilitating further processing, we are organising the dataframe to reorder the columns in a way that will be more manageable for the further steps

In [12]:
# general information on each game is put in the first rows, then we have the features for the home and the away team, and finally the odds

df = df[['match_id', 'season', 'date', 'away_team_id', 'away_team_name',
       'home_team_id', 'home_team_name',
       # feats home
       'home_team_rating','home_won_contest', 'home_possession_percentage', 'home_total_throws',
       'home_blocked_scoring_att', 'home_total_scoring_att',
       'home_total_tackle', 'home_aerial_won', 'home_aerial_lost','home_won_corners',
       'home_shot_off_target', 'home_ontarget_scoring_att',
       'home_total_offside', 'home_post_scoring_att', 'home_att_pen_goal',
       'home_penalty_save', 'HF', 'HY','HR', 'home_pass', 'goalkeeper_home_player_rating',
       'defender_home_player_rating', 'midfielder_home_player_rating', 'forward_home_player_rating', 'FTHG',
       # feats away
       'away_team_rating', 'away_won_contest', 'away_possession_percentage', 'away_total_throws',
       'away_blocked_scoring_att', 'away_total_scoring_att',
       'away_total_tackle', 'away_aerial_won', 'away_aerial_lost','away_won_corners',  
        'away_shot_off_target', 'away_ontarget_scoring_att', 
       'away_total_offside', 'away_post_scoring_att', 'away_att_pen_goal',
        'away_penalty_save', 'AF',  'AY', 'AR', 'away_pass', 'goalkeeper_away_player_rating',
       'defender_away_player_rating', 'midfielder_away_player_rating', 'forward_away_player_rating', 'FTAG',
       # target
       'FTR',
       # odds
       'B365H','B365D', 'B365A', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH',
       'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA']]

5. Replacing each feature by the mean of the 3 past matches for the given feature
In order to be able to create a model in the next notebook and to explore correlations, we first need to make sure that each row contains only data that is known at the time of the prediction.
As match statistics are only known after a match takes place, we decided to use the mean of the 3 previous matchs statistics. The assumption behind that process is also that taking more than one match into consideration would reduce the risk of bias that may happen if one time a team has an exceptional performance, and more generally this helps embed the general state of a team at a given time.

In [13]:
# Creating a dataframe that will include the mean for past matches
df_results = pd.DataFrame(columns=df.columns, index=df.index.values.tolist())

In [14]:
# This array will keep track of matches that are not processed due to lack of match data
index_unprocessed = []

In [15]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 1519
Data columns (total 76 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   match_id                       0 non-null      object
 1   season                         0 non-null      object
 2   date                           0 non-null      object
 3   away_team_id                   0 non-null      object
 4   away_team_name                 0 non-null      object
 5   home_team_id                   0 non-null      object
 6   home_team_name                 0 non-null      object
 7   home_team_rating               0 non-null      object
 8   home_won_contest               0 non-null      object
 9   home_possession_percentage     0 non-null      object
 10  home_total_throws              0 non-null      object
 11  home_blocked_scoring_att       0 non-null      object
 12  home_total_scoring_att         0 non-null      object
 13  hom

In [17]:
# This function will be used to retrieve the statistics for the 3 past matches
# To avoid bias that may be caused by the status of a team (Home/Away) we took each time the stats for a given time
# in the same context (past matches as home playing team, or past matches as an away playing team)

def previous_results(id, n):

  df_home_last_home_results = pd.DataFrame()
  df_home_last_results = pd.DataFrame()
  df_away_last_away_results = pd.DataFrame()
  df_away_last_results = pd.DataFrame()
  df_last_results = pd.DataFrame()

  # team names
  home = list(df[df['match_id'] == id]['home_team_name'])[0]
  away = list(df[df['match_id'] == id]['away_team_name'])[0]
  date = list(df[df['match_id'] == id]['date'])[0]

  # home dataframes
  df_home_last_home_results = df[(df['home_team_name'] == home) & (df['date'] < date)][:]
  df_home_last_results = df[((df['home_team_name'] == home) | (df['away_team_name'] == home)) & (df['date'] < date)][:]

  # away dataframes
  df_away_last_away_results = df[(df['away_team_name'] == away) & (df['date'] < date)][:]
  df_away_last_results = df[((df['home_team_name'] == away) | (df['away_team_name'] == away)) & (df['date'] < date)][:]

  # confrontation dataframes
  df_last_results = df[(((df['home_team_name'] == home) & (df['away_team_name'] == away)) | ((df['home_team_name'] == away) & (df['away_team_name'] == home))) & (df['date'] < date)][:]

  # n last matchs (for home_last_results and away_last_results seulement)
  df_home_last_home_results = df_home_last_home_results.sort_values(by = ['date'], ascending = False)
  df_home_last_results = df_home_last_results.sort_values(by = ['date'], ascending = False)
  df_away_last_away_results = df_away_last_away_results.sort_values(by = ['date'], ascending = False)
  df_away_last_results = df_away_last_results.sort_values(by = ['date'], ascending = False)
  df_last_results = df_last_results.sort_values(by = ['date'], ascending = False)

  if len(df_home_last_home_results) < n:
    df_home_last_home_results = df_home_last_home_results
  else:
    df_home_last_home_results = df_home_last_home_results[0:n]

  if len(df_home_last_results) < n:
    df_home_last_results = df_home_last_results
  else:
    df_home_last_results = df_home_last_results[0:n]

  if len(df_away_last_away_results) < n:
    df_away_last_away_results = df_away_last_away_results
  else:
    df_away_last_away_results = df_away_last_away_results[0:n]
  
  if len(df_away_last_results) < n:
    df_away_last_results = df_away_last_results
  else:
    df_away_last_results = df_away_last_results[0:n]
  
  if len(df_last_results) < n:
    df_last_results  =df_last_results
  else:
    df_last_results = df_last_results[0:n]

  return df_home_last_home_results, df_home_last_results, df_away_last_away_results, df_away_last_results, df_last_results

In [16]:
#function that will be used to compute the means for each row
def add_prev_res(index,row,match_id,a,c):

  # we keep only the matches for which we have 3 matches available in the past
  if (len(a) > 2 and len(c) > 2):
    
    results_home_mean = a[['home_team_rating','home_won_contest', 'home_possession_percentage', 'home_total_throws',
       'home_blocked_scoring_att', 'home_total_scoring_att',
       'home_total_tackle', 'home_aerial_won', 'home_aerial_lost','home_won_corners',
       'home_shot_off_target', 'home_ontarget_scoring_att',
       'home_total_offside', 'home_post_scoring_att', 'home_att_pen_goal',
       'home_penalty_save', 'HF', 'HY','HR', 'home_pass', 'goalkeeper_home_player_rating',
       'defender_home_player_rating', 'midfielder_home_player_rating', 'forward_home_player_rating','FTHG']].mean()

   
    results_away_mean = c[['away_team_rating', 'away_won_contest', 'away_possession_percentage', 'away_total_throws',
       'away_blocked_scoring_att', 'away_total_scoring_att',
       'away_total_tackle', 'away_aerial_won', 'away_aerial_lost','away_won_corners',  
        'away_shot_off_target', 'away_ontarget_scoring_att', 
       'away_total_offside', 'away_post_scoring_att', 'away_att_pen_goal',
        'away_penalty_save', 'AF',  'AY', 'AR', 'away_pass', 'goalkeeper_away_player_rating',
       'defender_away_player_rating', 'midfielder_away_player_rating', 'forward_away_player_rating','FTAG']].mean()
  
    # variables that are general info about each match will remain the same in the new tm
    df_results[['match_id', 'season', 'date', 'away_team_id', 'away_team_name',
       'home_team_id', 'home_team_name']] = row[['match_id', 'season', 'date', 'away_team_id', 'away_team_name',
       'home_team_id', 'home_team_name']]

    # Home means
    df_results[['home_team_rating','home_won_contest', 'home_possession_percentage', 'home_total_throws',
       'home_blocked_scoring_att', 'home_total_scoring_att',
       'home_total_tackle', 'home_aerial_won', 'home_aerial_lost','home_won_corners',
       'home_shot_off_target', 'home_ontarget_scoring_att',
       'home_total_offside', 'home_post_scoring_att', 'home_att_pen_goal',
       'home_penalty_save', 'HF', 'HY','HR', 'home_pass', 'goalkeeper_home_player_rating',
       'defender_home_player_rating', 'midfielder_home_player_rating', 'forward_home_player_rating','FTHG']] = results_home_mean
    
    # Away means
    df_results[['away_team_rating', 'away_won_contest', 'away_possession_percentage', 'away_total_throws',
       'away_blocked_scoring_att', 'away_total_scoring_att',
       'away_total_tackle', 'away_aerial_won', 'away_aerial_lost','away_won_corners',  
        'away_shot_off_target', 'away_ontarget_scoring_att', 
       'away_total_offside', 'away_post_scoring_att', 'away_att_pen_goal',
        'away_penalty_save', 'AF',  'AY', 'AR', 'away_pass', 'goalkeeper_away_player_rating',
       'defender_away_player_rating', 'midfielder_away_player_rating', 'forward_away_player_rating','FTAG']] = results_away_mean

    # Other variables that don't require means

    df_results[['FTR', 'B365H','B365D', 'B365A', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH',
       'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA']] = row[['FTR', 'B365H','B365D', 'B365A', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH',
       'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA']]

    # S'il n'y a pas de nb de matchs suffisant sur les matchs antérieurs pr
    # calculer les moyennes, afficher le match id pr info. Ces matchs ne seront pas inclus dans le df
  else:
      index_unprocessed.append(index)
      #print(index)
      #print(f'Nb Résultats home: {len(a)}')
      #print(f'Nb Résultats away: {len(c)}')

In [18]:
for index, row in df.iterrows():
  match_id = row['match_id']
  a, b, c, d, e = previous_results(match_id, 3)
  add_prev_res(index,row,match_id,a,c)

In [19]:
# number of unprocessed matches due to insufficient past data
len(index_unprocessed)

95

In [20]:
# removing unprocessed rows
to_drop = df_results.index[index_unprocessed]
df_results.drop(to_drop, inplace=True)

In [21]:
df_results.shape

(1425, 76)

In [22]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1425 entries, 60 to 1519
Data columns (total 76 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   match_id                       1425 non-null   int64  
 1   season                         1425 non-null   object 
 2   date                           1425 non-null   object 
 3   away_team_id                   1425 non-null   int64  
 4   away_team_name                 1425 non-null   object 
 5   home_team_id                   1425 non-null   int64  
 6   home_team_name                 1425 non-null   object 
 7   home_team_rating               1425 non-null   float64
 8   home_won_contest               1425 non-null   float64
 9   home_possession_percentage     1425 non-null   float64
 10  home_total_throws              1425 non-null   float64
 11  home_blocked_scoring_att       1425 non-null   float64
 12  home_total_scoring_att         1425 non-null   

In [23]:
df_results.to_csv('df_results.csv')